<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/XLM_Roberta_Fine_tuning_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

xlm RoBERTa

NOBREAK - Machado

Batch size = 16

GPU T4x2

In [1]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.6.4
Uninstalling tensorflow-2.6.4:
  Successfully uninstalled tensorflow-2.6.4
tokenizers                            0.12.1
transformers                          4.20.1


In [2]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"
#!curl -L https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa --output "dump.txt"
    
!awk NF < dump.txt > kant.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  2696k      0 --:--:-- --:--:-- --:--:-- 2692k


In [3]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-02212350d3c18f88/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [6]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (612 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (563 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (774 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
tokenized_datasets["train"][4]

{'input_ids': [0, 292, 27974, 11927, 32, 2],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [8]:
#block_size = tokenizer.model_max_length
block_size = 32

exemplo de entrada

In [9]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [12]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

In [13]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./xlm_RoBERTa-Machado_Nobreak",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],    
)

In [14]:
#@title Step 13: Pre-training the Model
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11412
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3570
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.770200
1000,2.465700
1500,2.309100
2000,2.200800
2500,2.103800
3000,2.046600
3500,2.015900




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3570, training_loss=2.268316945308397, metrics={'train_runtime': 2368.3087, 'train_samples_per_second': 48.186, 'train_steps_per_second': 1.507, 'total_flos': 1882117362355200.0, 'train_loss': 2.268316945308397, 'epoch': 10.0})

In [15]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 3918
  Batch size = 16


Perplexity: 8.29


In [16]:
eval_results

{'eval_loss': 2.115046739578247,
 'eval_runtime': 40.9529,
 'eval_samples_per_second': 95.671,
 'eval_steps_per_second': 5.982,
 'epoch': 10.0}

In [17]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./xlm_RoBERTa-Machado_Nobreak")

Saving model checkpoint to ./xlm_RoBERTa-Machado_Nobreak
Configuration saved in ./xlm_RoBERTa-Machado_Nobreak/config.json
Model weights saved in ./xlm_RoBERTa-Machado_Nobreak/pytorch_model.bin


In [18]:
# Saving model on Wandb
import wandb
wandb.save('xlm_RoBERTa-Machado_Nobreak.h5')

[]

In [19]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./xlm_RoBERTa-Machado_Nobreak",
    tokenizer=tokenizer
)

loading configuration file ./xlm_RoBERTa-Machado_Nobreak/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./xlm_RoBERTa-Machado_Nobreak",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading configuration file ./xlm_RoBERTa-Machado_Nobreak/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./xlm_RoBERTa-Machado_Nobreak",
  "architectures": [
    "X

In [20]:
fill_mask("O rapaz olhou para o <mask> ")

[{'score': 0.12774516642093658,
  'token': 13343,
  'token_str': 'outro',
  'sequence': 'O rapaz olhou para o outro'},
 {'score': 0.0785096064209938,
  'token': 118427,
  'token_str': 'marido',
  'sequence': 'O rapaz olhou para o marido'},
 {'score': 0.0752451941370964,
  'token': 5,
  'token_str': '.',
  'sequence': 'O rapaz olhou para o.'},
 {'score': 0.07046205550432205,
  'token': 48682,
  'token_str': 'médico',
  'sequence': 'O rapaz olhou para o médico'},
 {'score': 0.061107899993658066,
  'token': 177122,
  'token_str': 'chão',
  'sequence': 'O rapaz olhou para o chão'}]

In [21]:
fill_mask("A moça olhou para o <mask> ")

[{'score': 0.20976918935775757,
  'token': 118427,
  'token_str': 'marido',
  'sequence': 'A moça olhou para o marido'},
 {'score': 0.11062201112508774,
  'token': 48682,
  'token_str': 'médico',
  'sequence': 'A moça olhou para o médico'},
 {'score': 0.09720515459775925,
  'token': 13343,
  'token_str': 'outro',
  'sequence': 'A moça olhou para o outro'},
 {'score': 0.06236657500267029,
  'token': 175194,
  'token_str': 'céu',
  'sequence': 'A moça olhou para o céu'},
 {'score': 0.04451904073357582,
  'token': 5,
  'token_str': '.',
  'sequence': 'A moça olhou para o.'}]

In [22]:
fill_mask("Comprou uma <mask> na loja. ")

[{'score': 0.3083058297634125,
  'token': 185350,
  'token_str': 'cadeira',
  'sequence': 'Comprou uma cadeira na loja.'},
 {'score': 0.13047072291374207,
  'token': 20533,
  'token_str': 'carta',
  'sequence': 'Comprou uma carta na loja.'},
 {'score': 0.10865946859121323,
  'token': 2349,
  'token_str': 'casa',
  'sequence': 'Comprou uma casa na loja.'},
 {'score': 0.03926897421479225,
  'token': 30092,
  'token_str': 'coisa',
  'sequence': 'Comprou uma coisa na loja.'},
 {'score': 0.0240776427090168,
  'token': 128869,
  'token_str': 'cousa',
  'sequence': 'Comprou uma cousa na loja.'}]

In [23]:
fill_mask("A mulher não é <mask>. ")

[{'score': 0.2285887449979782,
  'token': 91276,
  'token_str': 'bonita',
  'sequence': 'A mulher não é bonita.'},
 {'score': 0.048108577728271484,
  'token': 19328,
  'token_str': 'boa',
  'sequence': 'A mulher não é boa.'},
 {'score': 0.0342043898999691,
  'token': 96394,
  'token_str': 'rara',
  'sequence': 'A mulher não é rara.'},
 {'score': 0.02655733935534954,
  'token': 1646,
  'token_str': 'sua',
  'sequence': 'A mulher não é sua.'},
 {'score': 0.02627628669142723,
  'token': 55998,
  'token_str': 'linda',
  'sequence': 'A mulher não é linda.'}]

In [24]:
fill_mask("O homem não é <mask>. ")

[{'score': 0.08954934775829315,
  'token': 44277,
  'token_str': 'homem',
  'sequence': 'O homem não é homem.'},
 {'score': 0.032654717564582825,
  'token': 8634,
  'token_str': 'bom',
  'sequence': 'O homem não é bom.'},
 {'score': 0.027262814342975616,
  'token': 47100,
  'token_str': 'poeta',
  'sequence': 'O homem não é poeta.'},
 {'score': 0.025749171152710915,
  'token': 44053,
  'token_str': 'padre',
  'sequence': 'O homem não é padre.'},
 {'score': 0.022515011951327324,
  'token': 135671,
  'token_str': 'raro',
  'sequence': 'O homem não é raro.'}]

In [25]:
fill_mask("Ele é um bom <mask>. ")

[{'score': 0.3058183491230011,
  'token': 38260,
  'token_str': 'amigo',
  'sequence': 'Ele é um bom amigo.'},
 {'score': 0.2646445333957672,
  'token': 44277,
  'token_str': 'homem',
  'sequence': 'Ele é um bom homem.'},
 {'score': 0.08024843782186508,
  'token': 114789,
  'token_str': 'rapaz',
  'sequence': 'Ele é um bom rapaz.'},
 {'score': 0.02895958721637726,
  'token': 131187,
  'token_str': 'senhor',
  'sequence': 'Ele é um bom senhor.'},
 {'score': 0.021274592727422714,
  'token': 187397,
  'token_str': 'menino',
  'sequence': 'Ele é um bom menino.'}]

In [26]:
fill_mask("Ela é uma boa <mask>. ")

[{'score': 0.3547772765159607,
  'token': 55207,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.1365812122821808,
  'token': 52636,
  'token_str': 'mulher',
  'sequence': 'Ela é uma boa mulher.'},
 {'score': 0.09741304069757462,
  'token': 29956,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'},
 {'score': 0.06621638685464859,
  'token': 51047,
  'token_str': 'dama',
  'sequence': 'Ela é uma boa dama.'},
 {'score': 0.0393059141933918,
  'token': 179879,
  'token_str': 'criatura',
  'sequence': 'Ela é uma boa criatura.'}]

In [27]:
fill_mask("Faz de conta que ainda é <mask>. ")

[{'score': 0.25703299045562744,
  'token': 15234,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'},
 {'score': 0.23700043559074402,
  'token': 79426,
  'token_str': 'criança',
  'sequence': 'Faz de conta que ainda é criança.'},
 {'score': 0.078849196434021,
  'token': 96872,
  'token_str': 'jovem',
  'sequence': 'Faz de conta que ainda é jovem.'},
 {'score': 0.051295626908540726,
  'token': 187397,
  'token_str': 'menino',
  'sequence': 'Faz de conta que ainda é menino.'},
 {'score': 0.027760937809944153,
  'token': 65312,
  'token_str': 'pequena',
  'sequence': 'Faz de conta que ainda é pequena.'}]

In [28]:
fill_mask("Depois da tempestade vem a <mask>. ")

[{'score': 0.21876192092895508,
  'token': 146672,
  'token_str': 'chuva',
  'sequence': 'Depois da tempestade vem a chuva.'},
 {'score': 0.13171151280403137,
  'token': 27761,
  'token_str': 'lua',
  'sequence': 'Depois da tempestade vem a lua.'},
 {'score': 0.05547480285167694,
  'token': 23291,
  'token_str': 'noite',
  'sequence': 'Depois da tempestade vem a noite.'},
 {'score': 0.03295222669839859,
  'token': 15234,
  'token_str': 'tarde',
  'sequence': 'Depois da tempestade vem a tarde.'},
 {'score': 0.028334297239780426,
  'token': 2436,
  'token_str': 'vida',
  'sequence': 'Depois da tempestade vem a vida.'}]

In [29]:
fill_mask("Mais vale um pássaro na mão do que <mask> voando. ")

[{'score': 0.28452759981155396,
  'token': 193,
  'token_str': 'ir',
  'sequence': 'Mais vale um pássaro na mão do que ir voando.'},
 {'score': 0.17927347123622894,
  'token': 25629,
  'token_str': 'ficar',
  'sequence': 'Mais vale um pássaro na mão do que ficar voando.'},
 {'score': 0.07381939142942429,
  'token': 78038,
  'token_str': 'andar',
  'sequence': 'Mais vale um pássaro na mão do que andar voando.'},
 {'score': 0.06585749983787537,
  'token': 68378,
  'token_str': 'sair',
  'sequence': 'Mais vale um pássaro na mão do que sair voando.'},
 {'score': 0.06499272584915161,
  'token': 5747,
  'token_str': 'estar',
  'sequence': 'Mais vale um pássaro na mão do que estar voando.'}]

In [30]:
fill_mask("Não tinha <mask>, mas almoçou mesmo assim. ")

[{'score': 0.12600962817668915,
  'token': 5799,
  'token_str': 'nada',
  'sequence': 'Não tinha nada, mas almoçou mesmo assim.'},
 {'score': 0.07373160868883133,
  'token': 204382,
  'token_str': 'almoço',
  'sequence': 'Não tinha almoço, mas almoçou mesmo assim.'},
 {'score': 0.02894417941570282,
  'token': 40454,
  'token_str': 'chá',
  'sequence': 'Não tinha chá, mas almoçou mesmo assim.'},
 {'score': 0.027864348143339157,
  'token': 26216,
  'token_str': 'café',
  'sequence': 'Não tinha café, mas almoçou mesmo assim.'},
 {'score': 0.025886669754981995,
  'token': 123299,
  'token_str': 'medo',
  'sequence': 'Não tinha medo, mas almoçou mesmo assim.'}]

In [31]:
fill_mask("Bebeu um copo d'água, pois tinha <mask>. ")

[{'score': 0.09760361164808273,
  'token': 123299,
  'token_str': 'medo',
  'sequence': "Bebeu um copo d'água, pois tinha medo."},
 {'score': 0.0451083667576313,
  'token': 31201,
  'token_str': 'água',
  'sequence': "Bebeu um copo d'água, pois tinha água."},
 {'score': 0.042474593967199326,
  'token': 54998,
  'token_str': 'vontade',
  'sequence': "Bebeu um copo d'água, pois tinha vontade."},
 {'score': 0.036253057420253754,
  'token': 11721,
  'token_str': 'pouco',
  'sequence': "Bebeu um copo d'água, pois tinha pouco."},
 {'score': 0.02871454320847988,
  'token': 128909,
  'token_str': 'razão',
  'sequence': "Bebeu um copo d'água, pois tinha razão."}]

In [32]:
fill_mask("Sem saber, ele descobriu um <mask>. ")

[{'score': 0.10955964028835297,
  'token': 165801,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.04084990173578262,
  'token': 196545,
  'token_str': 'estranho',
  'sequence': 'Sem saber, ele descobriu um estranho.'},
 {'score': 0.04083447903394699,
  'token': 26018,
  'token_str': 'livro',
  'sequence': 'Sem saber, ele descobriu um livro.'},
 {'score': 0.022593477740883827,
  'token': 7766,
  'token_str': 'nome',
  'sequence': 'Sem saber, ele descobriu um nome.'},
 {'score': 0.019689474254846573,
  'token': 11721,
  'token_str': 'pouco',
  'sequence': 'Sem saber, ele descobriu um pouco.'}]

In [33]:
fill_mask("Pedro fez fortuna vendendo <mask>. ")

[{'score': 0.16004246473312378,
  'token': 27,
  'token_str': '...',
  'sequence': 'Pedro fez fortuna vendendo....'},
 {'score': 0.08328715711832047,
  'token': 36,
  'token_str': 'o',
  'sequence': 'Pedro fez fortuna vendendo o.'},
 {'score': 0.07162977010011673,
  'token': 5,
  'token_str': '.',
  'sequence': 'Pedro fez fortuna vendendo..'},
 {'score': 0.04892539605498314,
  'token': 10964,
  'token_str': 'tudo',
  'sequence': 'Pedro fez fortuna vendendo tudo.'},
 {'score': 0.04600413143634796,
  'token': 10,
  'token_str': 'a',
  'sequence': 'Pedro fez fortuna vendendo a.'}]

In [34]:
fill_mask("Sem medo de ser <mask>. ")

[{'score': 0.1286020129919052,
  'token': 33587,
  'token_str': 'feliz',
  'sequence': 'Sem medo de ser feliz.'},
 {'score': 0.09224138408899307,
  'token': 44053,
  'token_str': 'padre',
  'sequence': 'Sem medo de ser padre.'},
 {'score': 0.02730511873960495,
  'token': 57873,
  'token_str': 'mãe',
  'sequence': 'Sem medo de ser mãe.'},
 {'score': 0.025949090719223022,
  'token': 44277,
  'token_str': 'homem',
  'sequence': 'Sem medo de ser homem.'},
 {'score': 0.023238174617290497,
  'token': 22733,
  'token_str': 'pai',
  'sequence': 'Sem medo de ser pai.'}]

In [35]:
wandb.finish()

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▂▃▄▆▇███
train/global_step,▁▂▃▄▆▇███
train/learning_rate,█▇▆▄▃▂▁
train/loss,█▅▄▃▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
